In [ ]:
import pandas as pd
import seaborn as sns
from glob import glob
sns.set_theme(style="ticks", font="Linux Biolinum O", font_scale=1.2)

frames = [pd.read_csv(data) for data in glob('../data/operation-*.txt')]
for frame in frames:
    if 'num_participant' not in frame:
        frame['num_participant'] = 10000
data = pd.concat(frames, ignore_index=True)
data = data[data['comment'].isna()]

data = data[data['num_concurrent'] == 8]
data = data[(data['inner_k'] == 32) & (data['inner_n'] == 80) & (data['outer_k'] == 8) & (data['outer_n'] == 10)]

def protocol_operation(row):
    return f'{row["protocol"].capitalize()} - {row["operation"]}'

data["Protocol - Operation"] = data.apply(protocol_operation, axis=1)
data

In [ ]:
hue_order = ["entropy - put", "kademlia - put", "entropy - get", "kademlia - get", "entropy - repair", "kademlia - repair"]
hue_order = [f"{p} - {o}" for o in ("put", "get", "repair") for p in ("Entropy", "Kademlia")]
entropy_palette = sns.cubehelix_palette(rot=0., light=0.75, n_colors=4)
kademlia_palette = sns.cubehelix_palette(start=2.5, rot=0., dark=0.25, n_colors=4)
palette = [c for p in zip(entropy_palette, reversed(kademlia_palette)) for c in p]

sns.relplot(data=data,
    kind="line",
    x="num_participant", y="latency", hue="Protocol - Operation",
    markers=['o'] * 6, dashes=False, style='Protocol - Operation',
    hue_order=hue_order, palette=palette,
    height=2.56, aspect=2.) \
.set_xlabels("Number of Nodes") \
.set_ylabels("Latency (s)") \
.savefig("../../graphs/scale.pdf")